In [1]:
import pandas as pd

In [13]:
filename = "gas_prices_2021-01-06_2021-01-07_5%-sampling.csv"

# first two lines are comments
f = open(filename)
print(f.readline())
print(f.readline())

df = pd.read_csv(f, delimiter='\t')

# Sampling 5% of transactions

# from 2021-01-06 00:00:00 to 2021-01-07 00:00:00



In [14]:
df

,blockNum,txnID,gasPrice
0,11597689,0xa82153cd764e27093bcdcf84acca4237c9238798ecca...,72000000000
1,11597689,0xa524f8ce1be4b130ccf1e96a1734a970910a6122a644...,72000000000
2,11597689,0x431d533c15e1c29bcbe2a223516a20b857aeba6cd55d...,72000000000
3,11597689,0xe2933b59a5887f1179f3dc45dd0e16826625359f639d...,72000000000
4,11597689,0x52b5e886a6904489ac291198451d31196f69ea9114f5...,76650000000
...,...,...,...
64037,11604174,0x4932fe85b2ac55242b6895ba9ce455a73047baaae511...,102500000000
64038,11604174,0x4bb285b5ea18867ef690c015b3a05917118d81f5a8d0...,100000000000
64039,11604174,0x2b1d57251ba9f4d63e0472f3123685ba2cf738a9fff4...,99000000000
64040,11604174,0x73cd75493cd576a6c5959c72445985db3bc79c3a3e4d...,96000000000


In [17]:
# get minimum gas price for each block
df_mins = df.groupby("blockNum", as_index=True)["gasPrice"].min()
df_mins

blockNum
11597689     72000000000
11597690     70000000000
11597691    109523809523
11597692      1000000000
11597693     88000000000
                ...     
11604170     82000000000
11604171     95000000000
11604172     89000001459
11604173     81000000000
11604174     95000000000
Name: gasPrice, Length: 6359, dtype: int64

# How many of these (sampled) transactions have a gas price greater than the min of the next block?

In [33]:
# TODO: how to do without for-loop?
yes = 0
no = 0
for _, row in df.T.iteritems():
    if row['blockNum'] + 1 not in df_mins.index:
        continue
    if row['gasPrice'] > df_mins.loc[row['blockNum'] + 1]:
        yes += 1
    else:
        no += 1

In [35]:
yes / (yes + no)

0.7990965628032002

# TODO: statistical significance testing
There are several layers to this:
- the computed proportion is only representative of a sample
- whether or not a given txn has a gas price greater than the min of the next block, is approximated (since we don't have all txns in the next block, thus min is an approx)
- and, of course, we have to account for uncertainty given by small sample range (a few days); i.e. get more data

## Other things:
- We are assuming that all txns of at least gas price x will get mined. This isn't necessarily the case for several reasons, but I think it's a good enough assumption that is mostly true.
- There is probably a long-tail thing going on: when a txn wouldn't get mined in the next block (i.e. it's gas price is less than the min of the next block) then it probably won't get mined in the one after that -- these kinds of things probably result from a price spike.